# Implementation of the Viterbi algorithm


The Viterbi algorithm is a well-known optimization algorithm for hidden Markov models.


This notebook demonstrates a basic implementation.

----

## Intro

The Viterbi algorithm allows you to find the most probable sequence of hidden states, given some observations i.e.

$$ Q^*(O) = \text{argmax} P(Q|O)$$

for path Q, optimal path $Q^*$ and observations $O$

See e.g. [Suvorova et al 2016](https://arxiv.org/pdf/1606.02412.pdf) for an example astrophysical implementation.


---



In [15]:
import numpy as np
def viterbi(y,transition_matrix,emission_matrix,initial_state):


    """
    A function to implement the Viterbi algorithm for HMM optimization

    Inputs:
        y                    :: Sequence of observations. Vector(T), Int dtype
        transition_matrix    :: Matrix of transition probabilities. Array (K,K)
        emission_matrix      :: Matrix of emission probabilities. Array (K,N) for N unique observation states
        initial_states       :: Initial probabilities system occupies each hidden state. Vector(K)

    Outputs:
        path                 :: Estimated hidden state path 
        probability          :: Probability of the path

    Acknowledgements:  https://stackoverflow.com/a/49351064/1887919


    """

    # Dimensions
    K = transition_matrix.shape[0] # Number of unique hidden states
    T = len(y)                     # Number of observations


    # Initialise two empty tables 
    T1 = np.zeros((K, T), 'd') #Tracking table T1, with double precison type. This is equivalent to \delta, Eq 11 from Suvorova.
    T2 = np.zeros((K, T), 'B') #Tracking table T2, with unsigned byte type. This is equivalent to \Phi, Eq 11 from Suvorova.


    #1. Initialization

    T1[:,0] = initial_state * emission_matrix[:, y[0]]
    T2[:, 0] = 0


    #2. Iterate
    # Iterate through the observations, updating the tracking tables. 
    # Index from 1 since the 0th element has already been set in the initialisation
    for i in range(1,T):
        T1[:,i] = np.max(T1[:, i - 1] * transition_matrix.T * emission_matrix[np.newaxis, :, y[i]].T,1) # This is Eq. 15 fro Suvorova
        T2[:,i] = np.argmax(T1[:,i-1]*transition_matrix.T,1)                                            #...and Eq. 16 


    #3. Infer the most probable hidden state path
    path = np.zeros(T, 'B')
    path[-1] = np.argmax(T1[:,T-1]) # Index of best final state


    #Backtrack
    for i in reversed(range(1,T)):
        path[i-1] = T2[path[i],i]

    probability = max(T1[:,T-1]) 
    return path, probability




We can check this is working ok by comparing it to the [worked example on Wikipedia](https://en.wikipedia.org/wiki/Viterbi_algorithm#Example).

In this case there are two hidden states _"Healthy"_ and _"Fever"_ and 3 observation states _"normal"_, _"cold"_ and _"dizzy"_.

The sequence of observations is simply `(normal,cold,dizzy)`.

Lets solve this using the above function:

In [16]:
obs = ["normal", "cold", "dizzy"] # observations
y = [0,1,2]                       # integer dtype corresponding to observation states
transition_matrix = np.array([[0.7, 0.3], [0.4, 0.6]])       # probabilities of transitions from one hidden state to another. From Wiipedia example 
emission_matrix = np.array([[0.5, 0.4,0.1], [0.1,0.3, 0.6]]) # probabilities of observations GIVEN a hidden state. From Wiipedia example 
initial_state = [0.6,0.4] #initial state probabilities

path,probability = viterbi(y,transition_matrix,emission_matrix,initial_state)


In [17]:
path

array([0, 0, 1], dtype=uint8)

This is equivalent to _"Healthy"_,_"Healthy"_, _"Fever"_ which agrees with the Wiki example

In [19]:
probability

0.01512

Which also agrees ✅